In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

In [ ]:
# Define the ResNet150 model
def residual_block(x, filters, kernel_size=3, stride=1, conv_shortcut=True, name=None):
    bn_axis = 3 if tf.keras.backend.image_data_format() == 'channels_last' else 1

    if conv_shortcut:
        shortcut = Conv2D(4 * filters, 1, strides=stride, name=name + '_0_conv')(x)
        shortcut = BatchNormalization(axis=bn_axis, name=name + '_0_bn')(shortcut)
    else:
        shortcut = x

    x = Conv2D(filters, 1, strides=stride, name=name + '_1_conv')(x)
    x = BatchNormalization(axis=bn_axis, name=name + '_1_bn')(x)
    x = Activation('relu', name=name + '_1_relu')(x)

    x = Conv2D(filters, kernel_size, padding='same', name=name + '_2_conv')(x)
    x = BatchNormalization(axis=bn_axis, name=name + '_2_bn')(x)
    x = Activation('relu', name=name + '_2_relu')(x)

    x = Conv2D(4 * filters, 1, name=name + '_3_conv')(x)
    x = BatchNormalization(axis=bn_axis, name=name + '_3_bn')(x)

    x = Add(name=name + '_add')([shortcut, x])
    x = Activation('relu', name=name + '_out')(x)
    return x


In [ ]:
def stack_blocks(x, filters, blocks, stride1=2, name=None):
    x = residual_block(x, filters, stride=stride1, name=name + '_block1')
    for i in range(2, blocks + 1):
        x = residual_block(x, filters, conv_shortcut=False, name=name + '_block' + str(i))
    return x

In [ ]:
def ResNet150(input_shape=(224, 224, 3)):
    img_input = Input(shape=input_shape)

    bn_axis = 3 if tf.keras.backend.image_data_format() == 'channels_last' else 1

    x = Conv2D(64, 7, strides=2, padding='same', name='conv1_conv')(img_input)
    x = BatchNormalization(axis=bn_axis, name='conv1_bn')(x)
    x = Activation('relu', name='conv1_relu')(x)
    x = tf.keras.layers.MaxPooling2D(3, strides=2, padding='same', name='pool1_pool')(x)

    x = stack_blocks(x, 64, 3, stride1=1, name='conv2')
    x = stack_blocks(x, 128, 8, name='conv3')
    x = stack_blocks(x, 256, 36, name='conv4')
    x = stack_blocks(x, 512, 3, name='conv5')

    model = Model(img_input, x, name='resnet150')
    return model


In [ ]:
# Define paths to your dataset
train_data_dir = '/content/drive/MyDrive/Glaucoma annoted/training'
test_data_dir = '/content/drive/MyDrive/Glaucoma annoted/validation'


In [ ]:
# Define batch size and number of epochs
batch_size = 16
epochs = 100
input_shape = (224, 224)  # Assuming images are resized to 224x224

print(f"Batch size: {batch_size}")

Batch size: 16


In [ ]:
# Data Augmentation and Image Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='binary'  # Since we use binary_crossentropy loss
)

Found 735 images belonging to 2 classes.


In [ ]:
# Flow validation images in batches using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='binary'
)

Found 116 images belonging to 2 classes.


In [ ]:
# Load the ResNet150 model
base_model = ResNet150(input_shape=(224, 224, 3))

In [ ]:
# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

print(f"Number of samples in training data: {train_generator.samples}")
print(f"Batch size: {batch_size}")

Number of samples in training data: 735
Batch size: 16


In [ ]:
# Calculate the total data count for training and validation datasets
total_train_samples = train_generator.samples
total_validation_samples = validation_generator.samples

print(f"Total training samples: {total_train_samples}")
print(f"Total validation samples: {total_validation_samples}")

Total training samples: 735
Total validation samples: 116


In [ ]:
# Add custom layers on top of ResNet150
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=150,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/150
45/45 [==============================] - 426s 9s/step - loss: 0.7173 - accuracy: 0.4882 - val_loss: 0.6850 - val_accuracy: 0.4375
Epoch 2/150
45/45 [==============================] - 453s 10s/step - loss: 0.6696 - accuracy: 0.5494 - val_loss: 0.6656 - val_accuracy: 0.4375
Epoch 3/150
45/45 [==============================] - 442s 10s/step - loss: 0.6402 - accuracy: 0.5869 - val_loss: 0.6864 - val_accuracy: 0.5714
Epoch 4/150
45/45 [==============================] - 407s 9s/step - loss: 0.6332 - accuracy: 0.6245 - val_loss: 0.6465 - val_accuracy: 0.4375
Epoch 5/150
45/45 [==============================] - 379s 8s/step - loss: 0.5875 - accuracy: 0.7636 - val_loss: 0.5594 - val_accuracy: 0.8393
Epoch 6/150
45/45 [==============================] - 419s 9s/step - loss: 0.5403 - accuracy: 0.7719 - val_loss: 0.5114 - val_accuracy: 0.9911
Epoch 7/150
45/45 [==============================] - 391s 9s/step - loss: 0.5081 - accuracy: 0.7955 - val_loss: 0.4633 - val_accuracy: 0.9643
Epoc

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation loss: {loss:.4f}, Validation accuracy: {accuracy:.4f}')

# Plot training history
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.tight_layout()
plt.show()